# Options Arbitrage

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from black_scholes import call_value, put_value, call_delta, put_delta, call_vega, put_vega
%matplotlib inline

In [2]:
def read_data(filename):
    df = pd.read_csv(filename, index_col=0)

    time_to_expiry = df.filter(like='TimeToExpiry')

    stock = df.filter(like='Stock')
    stock.columns = [stock.columns.str[-5:], stock.columns.str[:-6]]

    options = pd.concat((df.filter(like='-P'), df.filter(like='-C')), axis=1)
    options.columns = [options.columns.str[-3:], options.columns.str[:-4]]

    market_data = pd.concat((stock, options), axis=1)

    return time_to_expiry, market_data

In [3]:
# Read the market data
filename = 'Options Arbitrage.csv'
time_to_expiry, market_data = read_data(filename)

In [4]:
# Get a list of all instrument names including the stock, and of the options only
instrument_names = list(market_data.columns.get_level_values(0).unique())
print(instrument_names)

option_names = instrument_names[1:]
print(option_names)

['Stock', 'P60', 'P70', 'P80', 'C60', 'C70', 'C80']
['P60', 'P70', 'P80', 'C60', 'C70', 'C80']


In [5]:
# Add time_to_expiry to market_data
market_data['TTE'] = time_to_expiry['TimeToExpiry']

# Store timestamp in variable to prevent
# errors with multiplications and such
timestamp = market_data.index

# Set the Time to Expiry as Index
market_data = market_data.set_index('TTE')

In [6]:
# Create Empty Dictionaries
short_call_values = {}
long_call_values = {}
long_put_values = {}
short_put_values = {}
short_call_deltas = {}
long_call_deltas = {}
long_put_deltas = {}
short_put_deltas = {}
option_values = {}
option_deltas = {}

# Set known attributes
r = 0
sigma = 0.20

# Forloop to create new columns with Call/Put names
for option in option_names:
    # Retrieve K from the Option
    K = int(option[-2:])

    if 'C' in option:
        short_call_values[option] = []
        long_call_values[option] = []
        short_call_deltas[option] = []
        long_call_deltas[option] = []

        # Forloop to calculate short/long call values and deltas
        for time, stock_value in market_data.iterrows():
            short_call_values[option].append(call_value(
                stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            long_call_values[option].append(call_value(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            long_call_deltas[option].append(call_delta(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            short_call_deltas[option].append(-call_delta(
                stock_value['Stock', 'AskPrice'], K, time, r, sigma))

        option_values['Short Call', option] = short_call_values[option]
        option_values['Long Call', option] = long_call_values[option]
        option_deltas['Short Call', option] = short_call_deltas[option]
        option_deltas['Long Call', option] = long_call_deltas[option]

    if 'P' in option:
        long_put_values[option] = []
        short_put_values[option] = []
        long_put_deltas[option] = []
        short_put_deltas[option] = []

        # Forloop to calculate short/long put values and deltas
        for time, stock_value in market_data.iterrows():
            long_put_values[option].append(
                put_value(stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            short_put_values[option].append(
                put_value(stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            long_put_deltas[option].append(
                put_delta(stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            short_put_deltas[option].append(-put_delta(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))

        option_values['Long Put', option] = long_put_values[option]
        option_values['Short Put', option] = short_put_values[option]
        option_deltas['Long Put', option] = long_put_deltas[option]
        option_deltas['Short Put', option] = short_put_deltas[option]

In [7]:
# Create DataFrames with index market_data
option_values = pd.DataFrame(option_values, index=market_data.index)
option_deltas = pd.DataFrame(option_deltas, index=market_data.index)

# Sort the DataFrames
option_values = option_values.reindex(sorted(option_values.columns), axis=1)
option_deltas = option_deltas.reindex(sorted(option_deltas.columns), axis=1)

# Rounding
option_values = round(option_values, 2)

# Show DataFrames
print('Option Values')
display(option_values.head())
print('Option Deltas')
display(option_deltas.head())

Option Values


Long Call             Long Put              Short Call              \
               C60   C70   C80      P60   P70    P80        C60   C70   C80   
TTE                                                                           
0.911625     12.04  5.71  2.24     1.31  4.92  11.39      12.21  5.82  2.29   
0.911615     12.08  5.74  2.25     1.29  4.88  11.32      12.29  5.88  2.32   
0.911606     12.17  5.79  2.28     1.28  4.86  11.29      12.33  5.91  2.34   
0.911596     12.12  5.76  2.27     1.28  4.86  11.29      12.33  5.91  2.34   
0.911587     12.08  5.74  2.25     1.29  4.88  11.32      12.29  5.88  2.32   

         Short Put               
               P60   P70    P80  
TTE                              
0.911625      1.34  5.01  11.54  
0.911615      1.33  4.99  11.50  
0.911606      1.32  4.94  11.43  
0.911596      1.32  4.96  11.47  
0.911587      1.33  4.99  11.50

Option Deltas


Long Call                      Long Put                      \
               C60       C70       C80       P60       P70       P80   
TTE                                                                    
0.911625  0.830169  0.558665  0.290582 -0.166116 -0.435503 -0.704329   
0.911615  0.831105  0.560126  0.291851 -0.164281 -0.432599 -0.701776   
0.911606  0.832963  0.563041  0.294394 -0.163369 -0.431150 -0.700497   
0.911596  0.832038  0.561584  0.293120 -0.163368 -0.431150 -0.700499   
0.911587  0.831108  0.560126  0.291847 -0.164278 -0.432599 -0.701779   

         Short Call                     Short Put                      
                C60       C70       C80       P60       P70       P80  
TTE                                                                    
0.911625  -0.833884 -0.564497 -0.295671  0.169831  0.441335  0.709418  
0.911615  -0.835719 -0.567401 -0.298224  0.168895  0.439874  0.708149  
0.911606  -0.836631 -0.568850 -0.299503  0.167037  0.436959  0.705606  
0.911596  -0.836632 -0.568850 -0.299501  0.167962  0.438416  0.706880  
0.911587  -0.835722 -0.567401 -0.298221  0.168892  0.439874  0.708153

In [8]:
# Create Columns for Black Scholes Value in the Data Set
# This is used for later calculations (algorithm and such)

for option in option_names:
    if "C" in option:
        market_data[option,
                    'Expected AskPrice'] = option_values['Short Call', option]
        market_data[option,
                    'Expected BidPrice'] = option_values['Long Call', option]
        market_data[option,
                    'Delta Short'] = option_deltas['Short Call', option].values
        market_data[option,
                    'Delta Long'] = option_deltas['Long Call', option].values

    elif "P" in option:
        market_data[option,
                    'Expected AskPrice'] = option_values['Short Put', option]
        market_data[option,
                    'Expected BidPrice'] = option_values['Long Put', option]
        market_data[option,
                    'Delta Short'] = option_deltas['Short Put', option].values
        market_data[option,
                    'Delta Long'] = option_deltas['Long Put', option].values

# Sort Columns
market_data = market_data.reindex(sorted(market_data.columns), axis=1)

In [9]:
def option_opportunities(option):
    '''
    This function gives arbitrage opportunities based on whether the price
    of the option is too high or too low. The results are used to 'eyeball' 
    if our final results match what this function displays. This works for 
    all Calls and Puts.
    '''
    if "C" in option:
        expected1 = market_data[option][(market_data[option, 'BidPrice'] - market_data[option,
                                                                                       'Expected AskPrice']) >= 0.10].drop('Expected BidPrice', axis=1)
        expected2 = market_data[option][(market_data[option, 'Expected BidPrice'] -
                                         market_data[option, 'AskPrice']) >= 0.10].drop('Expected AskPrice', axis=1)

    elif "P" in option:
        expected1 = market_data[option][(market_data[option, 'BidPrice'] - market_data[option,
                                                                                       'Expected AskPrice']) >= 0.10].drop('Expected BidPrice', axis=1)
        expected2 = market_data[option][(market_data[option, 'Expected BidPrice'] -
                                         market_data[option, 'AskPrice']) >= 0.10].drop('Expected AskPrice', axis=1)

    print('BidPrice is at least 0.10 higher than Expected AskPrice for Option ' + option)
    display(expected1)
    print('AskPrice is at least 0.10 lower than Expected BidPrice for Option ' + option)
    display(expected2)
    print('The amount of trades are', len(expected1) + len(expected2))

option_opportunities('C80')

BidPrice is at least 0.10 higher than Expected AskPrice for Option C80


,AskPrice,AskVolume,BidPrice,BidVolume,Delta Long,Delta Short,Expected AskPrice
TTE,,,,,,,
0.910616,2.47,22.0,2.40,161.0,0.287906,-0.294257,2.28
0.908790,2.49,19.0,2.41,19.0,0.292730,-0.296557,2.30
0.863975,2.34,21.0,2.27,15.0,0.282434,-0.288892,2.17
0.846851,2.29,14.0,2.22,18.0,0.281149,-0.285046,2.11


AskPrice is at least 0.10 lower than Expected BidPrice for Option C80


,AskPrice,AskVolume,BidPrice,BidVolume,Delta Long,Delta Short,Expected BidPrice
TTE,,,,,,,
0.906355,2.03,17.0,1.96,19.0,0.282249,-0.288574,2.14
0.890925,1.70,122.0,1.63,21.0,0.258776,-0.262483,1.87
0.869368,1.31,86.0,1.25,19.0,0.218205,-0.222875,1.45
0.847146,1.96,143.0,1.88,19.0,0.279899,-0.285090,2.06
0.831088,2.43,14.0,2.35,16.0,0.324083,-0.329540,2.54
0.755689,3.13,22.0,3.05,21.0,0.388099,-0.395452,3.23
0.754014,3.09,95.0,3.00,18.0,0.386434,-0.392318,3.21


The amount of trades are 11


In [10]:
# Create a Dictionary with Timestamp and Time to Expiry
# Index of market_data was changed earlier to time to expiry
trades = {('Timestamp', ''): timestamp,
          ('Time to Expiry', ''): market_data.index}

# Forloop that adds columns for the Call/Put Positions and Deltas
# Global function is a changing variable name based on the option
# For option C60 it will create a variable named positions_call_C60
for option in option_names:

    if 'C' in option:
        trades['Call Position', option] = []
        trades['Call Delta', option] = []
        globals()['positions_call_' + option] = 0

    if 'P' in option:
        trades['Put Position', option] = []
        trades['Put Delta', option] = []
        globals()['positions_put_' + option] = 0

In [11]:
# Forloop over the rows of market_data
for time, data in market_data.iterrows():

    max_delta = min(data['Stock', 'AskVolume'], data['Stock', 'BidVolume'])

    # Forloop over the option_names with conditions
    # if-statements if Call or Put + if Short/Long in Call or Put
    for option in option_names:

        if 'C' in option:

            # Short Call
            if (data[option, 'BidPrice'] - data[option, 'Expected AskPrice']) >= 0.10:
                short_call_volume = data[option, 'BidVolume']
                long_call_volume = 0

            # Long Call
            elif (data[option, 'Expected BidPrice'] - data[option, 'AskPrice']) >= 0.10:
                long_call_volume = data[option, 'AskVolume']
                short_call_volume = 0

            else:
                long_call_volume = short_call_volume = 0

            call_trade = long_call_volume - short_call_volume

            # Define variable, as set earlier. Note the first position is set to zero otherwise
            # One would get an error here since the variable is then not yet defined.
            globals()['positions_call_' + option] = call_trade + \
                globals()['positions_call_' + option]

            # Add Positions (cumulative)
            trades['Call Position', option].append(
                globals()['positions_call_' + option])

            if globals()['positions_call_' + option] >= 0:
                long_call_delta = data[option, 'Delta Long']
                short_call_delta = 0

            elif globals()['positions_call_' + option] < 0:
                short_call_delta = data[option, 'Delta Short']
                long_call_delta = 0

            # Add Deltas (cumulative)
            trades['Call Delta', option].append(
                abs(globals()['positions_call_' + option]) * (long_call_delta + short_call_delta))

        if 'P' in option:

            # Short Put
            if (data[option, 'BidPrice'] - data[option, 'Expected AskPrice']) >= 0.10:
                short_put_volume = data[option, 'BidVolume']
                long_put_volume = 0

            # Long Put
            elif (data[option, 'Expected BidPrice'] - data[option, 'AskPrice']) >= 0.10:
                long_put_volume = data[option, 'AskVolume']
                short_put_volume = 0

            else:
                long_put_volume = short_put_volume = 0

            put_trade = long_put_volume - short_put_volume

            globals()['positions_put_' + option] = put_trade + \
                globals()['positions_put_' + option]

            trades['Put Position', option].append(
                globals()['positions_put_' + option])

            if globals()['positions_put_' + option] >= 0:
                long_put_delta = data[option, 'Delta Long']
                short_put_delta = 0

            elif globals()['positions_put_' + option] < 0:
                short_put_delta = data[option, 'Delta Short']
                long_put_delta = 0

            trades['Put Delta', option].append(
                abs(globals()['positions_put_' + option]) * (long_put_delta + short_put_delta))

In [12]:
# Create DataFrame with Index Timestamp
trades = pd.DataFrame(trades).set_index('Timestamp')

# Sort Columns
trades = trades.reindex(sorted(trades.columns), axis=1)

# Calculate Total Option Delta (based on sorted columns)
trades['Total Option Delta', ''] = np.sum(
    trades['Call Delta'], axis=1) + np.sum(trades['Put Delta'], axis=1)

# Calculate Cumulative Stock Position (floored if positive, ceiled if negative)
trades['Stock Position', 'Stock'] = -np.where(trades['Total Option Delta', ''] >= 0, np.floor(
    trades['Total Option Delta', '']), np.ceil(trades['Total Option Delta', '']))

# Calculate remaining option delta (that remains unhedged)
# This delta is included in the Total Option Delta again which ensures
# It always remains below zero
trades['Remaining Option Delta', ''] = trades['Total Option Delta',
                                              ''] + trades['Stock Position', 'Stock']

# Show DataFrame
trades.tail()

Call Delta                         Call Position         \
                            C60         C70         C80           C60    C70   
Timestamp                                                                      
2018-02-28 23:35:00  281.812848  375.708432  117.148014         305.0  532.0   
2018-02-28 23:40:00  281.646451  375.009738  116.724180         305.0  532.0   
2018-02-28 23:45:00  281.646800  375.010163  116.723895         305.0  532.0   
2018-02-28 23:50:00  281.647150  375.010589  116.723609         305.0  532.0   
2018-02-28 23:55:00  281.479734  374.309980  116.299862         305.0  532.0   

                           Put Delta                        Put Position  \
                       C80       P60        P70         P80          P60   
Timestamp                                                                  
2018-02-28 23:35:00  286.0 -7.017632 -10.388177  247.964456         95.0   
2018-02-28 23:40:00  286.0 -7.017525 -10.388147  248.586868         95.0   
2018-02-28 23:45:00  286.0 -7.017417 -10.388117  248.587287         95.0   
2018-02-28 23:50:00  286.0 -7.017310 -10.388087  248.587707         95.0   
2018-02-28 23:55:00  286.0 -7.118870 -10.481772  249.209993         95.0   

                                 Time to Expiry Total Option Delta  \
                      P70    P80                                     
Timestamp                                                            
2018-02-28 23:35:00  36.0 -420.0       0.750038        1005.227940   
2018-02-28 23:40:00  36.0 -420.0       0.750029        1004.561566   
2018-02-28 23:45:00  36.0 -420.0       0.750019        1004.562612   
2018-02-28 23:50:00  36.0 -420.0       0.750010        1004.563657   
2018-02-28 23:55:00  36.0 -420.0       0.750000        1003.698927   

                    Stock Position Remaining Option Delta  
                             Stock                         
Timestamp                                                  
2018-02-28 23:35:00        -1005.0               0.227940  
2018-02-28 23:40:00        -1004.0               0.561566  
2018-02-28 23:45:00        -1004.0               0.562612  
2018-02-28 23:50:00        -1004.0               0.563657  
2018-02-28 23:55:00        -1003.0               0.698927

In [13]:
# Create trades_diff dataframe that gives all actual trades (not positions)
# Also drop columns that are not required for PnL calculations and such
trades_diff = trades.diff()[1:].drop(
    ['Call Delta', 'Put Delta', 'Time to Expiry', 'Total Option Delta', 'Remaining Option Delta'], axis=1)

# Drop the 'Call Position','Put Position' and 'Stock Position' top level
# Makes forlooping easier
trades_diff.columns = trades_diff.columns.droplevel(level=0)

# Since positions are not neccesarily zero at the last timestamp, final positions are calculated to be able to valuate these
final_positions = trades[-1:].drop(['Call Delta', 'Put Delta', 'Time to Expiry',
                                    'Total Option Delta', 'Remaining Option Delta'], axis=1)

final_positions.columns = final_positions.columns.droplevel(level=0)

# Show DataFrames
print('Actual Trades/Volumes')
display(trades_diff.head())

print("Final Positions that we currently 'own'")
display(final_positions)

Actual Trades/Volumes


,C60,C70,C80,P60,P70,P80,Stock
Timestamp,,,,,,,
2018-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Final Positions that we currently 'own'


,C60,C70,C80,P60,P70,P80,Stock
Timestamp,,,,,,,
2018-02-28 23:55:00,305.0,532.0,286.0,95.0,36.0,-420.0,-1003.0


In [14]:
# Including Timestamp again to match trades_diff index
market_data['Timestamp'] = timestamp
market_data = market_data.set_index('Timestamp')

In [15]:
# Create Dataframe with index market_data (timestamp)
cashflow_dataframe = pd.DataFrame(index=market_data.index[1:])

# Forloop on all instruments (including stock) to calculate PnL
for instrument in instrument_names:

    Instrument_AskPrice = market_data[instrument, 'AskPrice'][1:]
    Instrument_BidPrice = market_data[instrument, 'BidPrice'][1:]

    cashflow_dataframe[instrument] = np.where(trades_diff[instrument] >= 0,
                                              trades_diff[instrument] * -
                                              Instrument_AskPrice,
                                              trades_diff[instrument] * -Instrument_BidPrice)

# Show DataFrame & PnL
display(cashflow_dataframe.head())
total_cashflow = cashflow_dataframe.sum().sum()

print('The total Cashflow is: €', round(total_cashflow, 2))

,Stock,P60,P70,P80,C60,C70,C80
Timestamp,,,,,,,
2018-01-01 00:10:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2018-01-01 00:15:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2018-01-01 00:20:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2018-01-01 00:25:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2018-01-01 00:30:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


The total Cashflow is: € 68985.54


In [16]:
# Cumulative Cashflows
cashflow_cumulative = {}

for column in cashflow_dataframe.columns:

    cashflow_cumulative[column] = cashflow_dataframe[column].cumsum()

cashflow_cumulative = pd.DataFrame(cashflow_cumulative)

# Show Cumulative Cashflow
display(cashflow_cumulative.tail())

# Checking for Match
print('This number should match the above number: €',
      round(cashflow_cumulative[-1:].sum().sum(), 2))

,Stock,P60,P70,P80,C60,C70,C80
Timestamp,,,,,,,
2018-02-28 23:35:00,71187.00,-116.36,762.03,4326.29,-3141.69,-3354.85,-525.08
2018-02-28 23:40:00,71111.05,-116.36,762.03,4326.29,-3141.69,-3354.85,-525.08
2018-02-28 23:45:00,71111.05,-116.36,762.03,4326.29,-3141.69,-3354.85,-525.08
2018-02-28 23:50:00,71111.05,-116.36,762.03,4326.29,-3141.69,-3354.85,-525.08
2018-02-28 23:55:00,71035.20,-116.36,762.03,4326.29,-3141.69,-3354.85,-525.08


This number should match the above number: € 68985.54


In [17]:
# Create a new dataframe of trades with most columns dropped
trades_minimal = trades.drop(['Call Delta', 'Put Delta', 'Time to Expiry', 'Total Option Delta',
                              'Remaining Option Delta'], axis=1)

trades_minimal.columns = trades_minimal.columns.droplevel(level=0)

# Create Dataframe with market_data as index
valuation_dataframe = pd.DataFrame(index=market_data.index)

# Forloop to calculate valuations on every timestamp
for instrument in instrument_names:

    if 'C' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

    if 'P' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

    if 'S' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

# Show DataFrame & Calculate total valuation
display(valuation_dataframe.tail())
total_valuation = valuation_dataframe[-1:].sum().sum()

print("Total valuation of our Position is currently: €", round(total_valuation, 2))

,Stock,P60,P70,P80,C60,C70,C80
Timestamp,,,,,,,
2018-02-28 23:35:00,-76329.75,46.55,93.24,-3238.2,4971.50,4500.72,986.70
2018-02-28 23:40:00,-76253.80,45.60,91.80,-3234.0,4974.55,4516.68,975.26
2018-02-28 23:45:00,-76253.80,46.55,90.36,-3246.6,4962.35,4495.40,978.12
2018-02-28 23:50:00,-76253.80,43.70,90.00,-3234.0,4974.55,4532.64,989.56
2018-02-28 23:55:00,-76077.55,44.65,90.36,-3280.2,4944.05,4500.72,975.26


Total valuation of our Position is currently: € -68802.71


In [18]:
# Create Empty DataFrame
blackscholes_dataframe = {}

# Create Columns based on Option Names
for option in option_names:

    if 'C' in option:
        blackscholes_dataframe[option] = []

    if 'P' in option:
        blackscholes_dataframe[option] = []

# Forloop that calculates the margins and thus profits
for time, data in market_data.iterrows():
    
    for option in option_names:

        if "C" in option:
            margin1 = data[option, 'BidPrice'] - data[option, 'Expected AskPrice']
            margin2 = data[option, 'Expected BidPrice'] - data[option, 'AskPrice']

            if margin1 > 0.10:
                blackscholes_dataframe[option].append(margin1)

            elif margin2 > 0.10:
                blackscholes_dataframe[option].append(margin2)

            else:
                blackscholes_dataframe[option].append(0)

        elif "P" in option:
            margin1 = data[option, 'BidPrice'] - data[option, 'Expected AskPrice']
            margin2 = data[option, 'Expected BidPrice'] - data[option, 'AskPrice']

            if margin1 > 0.10:
                blackscholes_dataframe[option].append(margin1)

            elif margin2 > 0.10:
                blackscholes_dataframe[option].append(margin2)

            else:
                blackscholes_dataframe[option].append(0)

# Create DataFrame with index of market_data
blackscholes_dataframe = pd.DataFrame(blackscholes_dataframe, index=market_data.index)

# Calculate Black_Scholes Profit
total_blackscholes = (abs(trades_diff).drop('Stock', axis=1) * blackscholes_dataframe).sum().sum()
print('The total profit from Black Scholes is: €',round(total_blackscholes,2))

The total profit from Black Scholes is: € 547.08


In [19]:
# Blackscholes Dataframe times Volumes
blackscholes_dataframe = pd.DataFrame(
    abs(trades_diff).drop('Stock', axis=1) * blackscholes_dataframe)

# Cumulative Blackscholes
blackscholes_cumulative = {}

for column in blackscholes_dataframe.columns:

    blackscholes_cumulative[column] = blackscholes_dataframe[column].cumsum()

blackscholes_cumulative = pd.DataFrame(blackscholes_cumulative)

# Show Dataframe
blackscholes_cumulative.tail()

,C60,C70,C80,P60,P70,P80
Timestamp,,,,,,
2018-02-28 23:35:00,61.95,77.29,88.98,14.33,135.36,169.17
2018-02-28 23:40:00,61.95,77.29,88.98,14.33,135.36,169.17
2018-02-28 23:45:00,61.95,77.29,88.98,14.33,135.36,169.17
2018-02-28 23:50:00,61.95,77.29,88.98,14.33,135.36,169.17
2018-02-28 23:55:00,61.95,77.29,88.98,14.33,135.36,169.17


In [20]:
# Last PnL in the Table should match the below value
print('The total profit generated from the Option Arbitrage strategy is: €',
      round(total_cashflow + total_valuation + total_blackscholes, 2))

The total profit generated from the Option Arbitrage strategy is: € 729.91
